In [1]:
import re
import time
import csv
import os
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
from scrapy import Selector
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys

from fake_useragent import UserAgent

<frozen importlib._bootstrap>:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
<frozen importlib._bootstrap>:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
<frozen importlib._bootstrap>:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
<frozen importlib._bootstrap>:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject


In [2]:
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.maximize_window()



====== WebDriver manager ======
Current google-chrome version is 106.0.5249
Get LATEST chromedriver version for 106.0.5249 google-chrome
Driver [C:\Users\user\.wdm\drivers\chromedriver\win32\106.0.5249.61\chromedriver.exe] found in cache
<ipython-input-2-0d9242eb10b9>:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [ ]:
driver.get('https://www.google.com/maps/search/burger,+frankfurt/@50.1216196,8.4964815,11z/data=!4m4!2m3!5m1!4e9!6e5')

In [ ]:
res = Selector(text=driver.page_source)

In [ ]:
cities = pd.read_csv('German Cities top 200 .csv', encoding= 'unicode_escape')

list_of_dict = list()
for city in cities['cities']:
    
    driver.get('https://www.google.com/search?q={}+zip+codes'.format(city))
    time.sleep(2)
    res = Selector(text=driver.page_source)
    zipcodes = res.css('div.bVj5Zb.FozYP::text').getall()
    
    if not zipcodes:
        zipcodes = res.css('#main > div:nth-child(5) > div > div > a > div > div > span > div::text').getall()
    print(zipcodes)
    
    count = 1
    for zipcode in zipcodes:
        if count < 350:
            try:
                driver.get('https://www.google.com/search?q={}+ restaurants {}'.format(zipcode,city))
                time.sleep(2)
#                 driver.find_element_by_css_selector('g-more-link > a').click()
                driver.find_element_by_css_selector('#Odp5De > div > div > div.ba9qXc > div:nth-child(3) > div > div:nth-child(1) > div > div.F7IIOc.xaNsfc > div').click()
                time.sleep(2)
                driver.find_element_by_css_selector('#filter_1 > div:nth-child(7) > span:nth-child(1) > span').click()
                time.sleep(2)
                
                names = list()
                addresses = list()

                while(1):
                    res = Selector(text=driver.page_source)
                    restaurant_names = res.css('div.rllt__details div.dbg0pd.eDIkBe span.OSrXXb::text').getall()
                    restaurant_addresses = res.css('a.vwVdIc.wzN8Ac.rllt__link.a-no-hover-decoration > div > div > div:nth-child(3)::text').getall()
                    # restaurant_urls = res.css('a.hfpxzc::attr(href)').getall()
                    # restaurant_names = res.css('a.hfpxzc::attr(aria-label)').getall()
#                     print('Names: ',restaurant_names)
#                     print('Addresses: ',restaurant_addresses)

                    names.extend(restaurant_names)
                    addresses.extend(restaurant_addresses)


                    next_page = res.css('#pnnext::attr(href)').get()

                    if next_page:

                        next_page = 'https://www.google.com' + next_page
                        driver.get(next_page)
                    else:
                        break

                print('RESTAURANT NAMES: ', names)
                print('RESTAURANT ADDRESSES: ', addresses)


                for name,address in zip(names,addresses):
                    print('Count: ', count)
                    count += 1
                    try:
                        print('Name: ',name)
                        print('address: ',address)
                        driver.get('https://www.google.com/maps/')
                        time.sleep(3)
                        WebDriverWait(driver,10).until(EC.visibility_of_element_located((By.XPATH, "(//input[@class='tactile-searchbox-input'])[1]"))).send_keys("{} {}".format(name,address))
                        time.sleep(5)
                        driver.find_element_by_css_selector("div.DgCNMb").click()
        #                 driver.find_element_by_css_selector("#searchbox-searchbutton").click()

                        temp=dict()
                        time.sleep(10)
                        res1 = Selector(text=driver.page_source)
                        address = res1.css('div.Io6YTe.fontBodyMedium::text').get()
                        address1 = address.split(',')
        #                 print('Address1: ', address1)
                        if len(address1) > 3:
                            address1 = address1[1:]

                        zipcode = address1[1].strip().split(" ")[0]
                    #     zipcode = address1[1].split(" ")[0]

                        town = address1[1].strip().split(" ")
        #                 if len(town) == 2:
        #                     town = town[0]
        #                 else:
                        town = " ".join(town[1:])
                        street = address1[0]
                        rating = res1.css('div.F7nice > span:nth-child(1) > span:nth-child(1) > span:nth-child(1)::text').get()
                        total_reviews = res1.css('.F7nice button.DkEaL::text').get()


#                         print('Name: ', name)
#                         print('Street: ', street)
#                         print('Zip Code: ', zipcode)
#                         print('Town: ', town)
#                         print('Rating: ', rating)
#                         print('Total Reviews: ', total_reviews)

                        temp = dict()
                        temp['name'] = name
                        temp['street'] = street
                        temp['zipcode'] = zipcode
                        temp['town'] = town
                        temp['rating'] = rating
                        temp['total_reviews'] = total_reviews
                        list_of_dict.append(temp)
                    except:
                        try:
#                             print('Name: ',name)
#                             print('address: ',address)
                            driver.get('https://www.google.com/maps/')
                            time.sleep(3)
                            WebDriverWait(driver,10).until(EC.visibility_of_element_located((By.XPATH, "(//input[@class='tactile-searchbox-input'])[1]"))).send_keys("{} {}".format(name,address))
                            time.sleep(5)
                            driver.find_element_by_css_selector("div.DgCNMb").click()
        #                     driver.find_element_by_css_selector("#searchbox-searchbutton").click()
                            temp=dict()
                            time.sleep(10)
                            restaurant_url = res1.css('a.hfpxzc::attr(href)').get()
#                             print(restaurant_url)
                            driver.get(restaurant_url)
                            time.sleep(2)
                            res1 = Selector(text=driver.page_source)
                            address = res1.css('div.Io6YTe.fontBodyMedium::text').get()
                            address1 = address.split(',')
#                             print('Address1: ', address1)
                            if len(address1) > 3:
                                address1 = address1[1:]

                            zipcode = address1[1].strip().split(" ")[0]
                        #     zipcode = address1[1].split(" ")[0]

                            town = address1[1].strip().split(" ")
            #                 if len(town) == 2:
            #                     town = town[0]
            #                 else:
                            town = " ".join(town[1:])
                            street = address1[0]
                            rating = res1.css('div.F7nice > span:nth-child(1) > span:nth-child(1) > span:nth-child(1)::text').get()
                            total_reviews = res1.css('.F7nice button.DkEaL::text').get()


#                             print('Name: ', name)
#                             print('Street: ', street)
#                             print('Zip Code: ', zipcode)
#                             print('Town: ', town)
#                             print('Rating: ', rating)
#                             print('Total Reviews: ', total_reviews)

                            temp = dict()
                            temp['name'] = name
                            temp['street'] = street
                            temp['zipcode'] = zipcode
                            temp['town'] = town
                            temp['rating'] = rating
                            temp['total_reviews'] = total_reviews
                            list_of_dict.append(temp)

                        except:
                            try:
#                                 print('Name: ',name)
#                                 print('address: ',address)

            #                     address = address.strip().split(" ")[0]
            #                     print('New address: ', address)
                                driver.get('https://www.google.com/maps/')
                                time.sleep(3)
                                WebDriverWait(driver,10).until(EC.visibility_of_element_located((By.XPATH, "(//input[@class='tactile-searchbox-input'])[1]"))).send_keys("{}".format(name))
                                time.sleep(5)
                                driver.find_element_by_css_selector("div.DgCNMb").click()

                                temp=dict()
                                time.sleep(10)
                                res1 = Selector(text=driver.page_source)
                                address = res1.css('div.Io6YTe.fontBodyMedium::text').get()
                                address1 = address.split(',')
#                                 print('Address1: ', address1)
                                if len(address1) > 3:
                                    address1 = address1[1:]

                                zipcode = address1[1].strip().split(" ")[0]
                        #     zipcode = address1[1].split(" ")[0]

                                town = address1[1].strip().split(" ")
                #                 if len(town) == 2:
                #                     town = town[0]
                #                 else:
                                town = " ".join(town[1:])
                                street = address1[0]
                                rating = res1.css('div.F7nice > span:nth-child(1) > span:nth-child(1) > span:nth-child(1)::text').get()
                                total_reviews = res1.css('.F7nice button.DkEaL::text').get()


#                                 print('Name: ', name)
#                                 print('Street: ', street)
#                                 print('Zip Code: ', zipcode)
#                                 print('Town: ', town)
#                                 print('Rating: ', rating)
#                                 print('Total Reviews: ', total_reviews)

                                temp = dict()
                                temp['name'] = name
                                temp['street'] = street
                                temp['zipcode'] = zipcode
                                temp['town'] = town
                                temp['rating'] = rating
                                temp['total_reviews'] = total_reviews
                                list_of_dict.append(temp)
                            except:
                                try:
                                    time.sleep(3)
                                    restaurant_url = res1.css('a.hfpxzc::attr(href)').get()
#                                     print(restaurant_url)
                                    driver.get(restaurant_url)
                                    time.sleep(5)
                                    temp=dict()

                                    res1 = Selector(text=driver.page_source)
                                    address = res1.css('div.Io6YTe.fontBodyMedium::text').get()
                                    try:
                                        address1 = address.split(',')
                                    except:
                                        address = res1.css('span.DkEaL::text').get()
                                        address1 = address.split(',')
#                                     print('Address1: ', address1)
                                    if len(address1) > 3:
                                        address1 = address1[1:]

                                    zipcode = address1[1].strip().split(" ")[0]
                                #     zipcode = address1[1].split(" ")[0]

                                    town = address1[1].strip().split(" ")
                    #                 if len(town) == 2:
                    #                     town = town[0]
                    #                 else:
                                    town = " ".join(town[1:])
                                    street = address1[0]
                                    rating = res1.css('div.F7nice > span:nth-child(1) > span:nth-child(1) > span:nth-child(1)::text').get()
                                    total_reviews = res1.css('.F7nice button.DkEaL::text').get()


#                                     print('Name: ', name)
#                                     print('Street: ', street)
#                                     print('Zip Code: ', zipcode)
#                                     print('Town: ', town)
#                                     print('Rating: ', rating)
#                                     print('Total Reviews: ', total_reviews)

                                    temp = dict()
                                    temp['name'] = name
                                    temp['street'] = street
                                    temp['zipcode'] = zipcode
                                    temp['town'] = town
                                    temp['rating'] = rating
                                    temp['total_reviews'] = total_reviews
                                    list_of_dict.append(temp)

                                except:
#                                     print('Name: ',name)
#                                     print('address: ',address)
                                    driver.get('https://www.google.com/maps/')
                                    time.sleep(3)
                                    WebDriverWait(driver,10).until(EC.visibility_of_element_located((By.XPATH, "(//input[@class='tactile-searchbox-input'])[1]"))).send_keys("{} {}".format(name,city))
                                    time.sleep(5)
                                    driver.find_element_by_css_selector("div.DgCNMb").click()
                #                     driver.find_element_by_css_selector("#searchbox-searchbutton").click()
                                    temp=dict()
                                    time.sleep(10)
        #                             restaurant_url = res1.css('a.hfpxzc::attr(href)').get()
        #                             print(restaurant_url)
        #                             driver.get(restaurant_url)
        #                             time.sleep(2)
                                    res1 = Selector(text=driver.page_source)
                                    address = res1.css('div.Io6YTe.fontBodyMedium::text').get()
                                    address1 = address.split(',')
#                                     print('Address1: ', address1)
                                    if len(address1) > 3:
                                        address1 = address1[1:]

                                    zipcode = address1[1].strip().split(" ")[0]
                                #     zipcode = address1[1].split(" ")[0]

                                    town = address1[1].strip().split(" ")
                    #                 if len(town) == 2:
                    #                     town = town[0]
                    #                 else:
                                    town = " ".join(town[1:])
                                    street = address1[0]
                                    rating = res1.css('div.F7nice > span:nth-child(1) > span:nth-child(1) > span:nth-child(1)::text').get()
                                    total_reviews = res1.css('.F7nice button.DkEaL::text').get()


                                    print('Name: ', name)
                                    print('Street: ', street)
                                    print('Zip Code: ', zipcode)
                                    print('Town: ', town)
                                    print('Rating: ', rating)
                                    print('Total Reviews: ', total_reviews)

                                    temp = dict()
                                    temp['name'] = name
                                    temp['street'] = street
                                    temp['zipcode'] = zipcode
                                    temp['town'] = town
                                    temp['rating'] = rating
                                    temp['total_reviews'] = total_reviews
                                    list_of_dict.append(temp)

                print('Temp: ',temp)
            except:
                pass
        else:
            break
#             driver.find_element_by_css_selector('button.B7V4Ld').click()
        

['10115', '10117', '10119', '10176', '10178', '10179', '10243', '10245', '10247', '10249', '10315', '10317', '10318', '10319', '10365', '10367', '10369', '10405', '10407', '10409', '10435', '10437', '10439', '10551', '10553', '10555', '10557', '10559', '10585', '10587', '10589', '10623', '10625', '10627', '10629', '10707', '10709', '10711', '10713', '10715', '10717', '10719', '10777', '10779', '10781', '10783', '10785', '10787', '10789', '10823', '10825', '10827', '10829', '10961', '10963', '10965', '10967', '10969', '10997', '10999', '12043', '12045', '12047', '12049', '12051', '12053', '12055', '12057', '12059', '12099', '12101', '12103', '12105', '12107', '12109', '12157', '12159', '12161', '12163', '12165', '12167', '12169', '12203', '12205', '12207', '12209', '12247', '12249', '12277', '12279', '12305', '12307', '12309', '12347', '12349', '12351', '12353', '12355', '12357', '12359', '12435', '12437', '12439', '12459', '12487', '12489', '12524', '12526', '12527', '12555', '12557', 

<ipython-input-4-1ff47a5cc5c0>:22: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_css_selector('#Odp5De > div > div > div.ba9qXc > div:nth-child(3) > div > div:nth-child(1) > div > div.F7IIOc.xaNsfc > div').click()
<ipython-input-4-1ff47a5cc5c0>:24: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_css_selector('#filter_1 > div:nth-child(7) > span:nth-child(1) > span').click()


RESTAURANT NAMES:  ['Seaside - Fish & Seafood Bar', 'Minty Restaurant', 'Cô Chu', 'Bonfini', 'SpeiseKombinat', 'Schnitzelei Mitte', 'La Buvette Bistrot Mitte', 'Golden Rice', 'royals & rice Berlin', 'Pascarella', 'restaurant gärtnerei', 'Xinh Xinh Vietnamese Food', '6868 Restaurant', 'Due Fratelli', 'DJIMALAYA - Hummus & Grill', 'CRAFTERIE Restaurant & Bar', 'Hung Anh - Vietnamese & Japanese', 'Lokal', 'PHO - Noodlebar (Mitte)', 'Kopps', 'Lila Vietnam fine cuisine', 'Salamat', 'Chelany-Mitte Restaurant', 'Rutz', 'Creasian', 'Mantée', 'noto Bar & Restaurant', 'HOONGY', 'heimlichTreu', 'einsunternull', 'Trattoria Pizzeria Bocca Del Buon Gusto', 'CORDO', 'DASHI', 'Nomad Berlin - Breakfast & Dinner', 'Kebap 63', 'Ristorante Nuovo Firenze', 'Restaurant Esszimmer', "Jasper's Mitte | Restaurant | Bar | Hotel", 'Gaststätte Sophieneck', 'Ristorante Cavallino Rosso', 'Gambrinus', 'MontRaw Restaurant', 'RELAX - FOOD, DRINKS & YOU', 'Dada Falafel', 'Poveracci', 'Yarok', 'Kin Za - Georgisches Resta

<ipython-input-4-1ff47a5cc5c0>:66: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_css_selector("div.DgCNMb").click()


Count:  2
Name:  Minty Restaurant
address:  Schönhauser Allee 134
Count:  3
Name:  Cô Chu
address:  Invalidenstraße 29
Count:  4
Name:  Bonfini
address:  Chausseestraße 15
Count:  5
Name:  SpeiseKombinat
address:  Chausseestraße 116


In [ ]:
len(list_of_dict)

In [ ]:
df = pd.DataFrame(list_of_dict)
df.to_csv('data31.csv',index=False)

In [ ]:
pd.read_csv('')

# ORIGNAL

In [ ]:
list_of_dict = list()
cities = pd.read_csv('German Cities top 200 .csv', encoding= 'unicode_escape')

for city in cities['cities'][6:8]:
    
    try:
        driver.get('https://www.google.com/search?q={}+restaurants'.format(city))
        time.sleep(2)
        driver.find_element_by_css_selector('#Odp5De > div > div > div.tJicRe > div:nth-child(3) > div > div:nth-child(1) > div > div.F7IIOc.xaNsfc > div').click()
        time.sleep(2)
        driver.find_element_by_css_selector('#filter_1 > div:nth-child(3) > span:nth-child(1) > span:nth-child(1)').click()
        time.sleep(2)
            
#         res = Selector(text=driver.page_source)
#         restaurant_names = res.css('div.rllt__details div.dbg0pd.eDIkBe span.OSrXXb::text').getall()
#         restaurant_addresses = res.css('a.vwVdIc.wzN8Ac.rllt__link.a-no-hover-decoration > div > div > div:nth-child(3)::text').getall()
#         # restaurant_urls = res.css('a.hfpxzc::attr(href)').getall()
#         # restaurant_names = res.css('a.hfpxzc::attr(aria-label)').getall()
#         print('Names: ',restaurant_names)
#         print('Addresses: ',restaurant_addresses)
        
        names = list()
        addresses = list()

        while(1):
            res = Selector(text=driver.page_source)
            restaurant_names = res.css('div.rllt__details div.dbg0pd.eDIkBe span.OSrXXb::text').getall()
            restaurant_addresses = res.css('a.vwVdIc.wzN8Ac.rllt__link.a-no-hover-decoration > div > div > div:nth-child(3)::text').getall()
            # restaurant_urls = res.css('a.hfpxzc::attr(href)').getall()
            # restaurant_names = res.css('a.hfpxzc::attr(aria-label)').getall()
#                     print('Names: ',restaurant_names)
#                     print('Addresses: ',restaurant_addresses)

            names.extend(restaurant_names)
            addresses.extend(restaurant_addresses)


            next_page = res.css('#pnnext::attr(href)').get()

            if next_page:

                next_page = 'https://www.google.com' + next_page
                driver.get(next_page)
            else:
                break

        print('RESTAURANT NAMES: ', names)
        print('RESTAURANT ADDRESSES: ', addresses)

        print(len(names))

        for name,address in zip(names,addresses):
            try:
#                 print('Name: ',name)
#                 print('address: ',address)
                driver.get('https://www.google.com/maps/')
                time.sleep(3)
                WebDriverWait(driver,10).until(EC.visibility_of_element_located((By.XPATH, "(//input[@class='tactile-searchbox-input'])[1]"))).send_keys("{} {}".format(name,address))
                time.sleep(5)
                driver.find_element_by_css_selector("div.DgCNMb").click()
#                 driver.find_element_by_css_selector("#searchbox-searchbutton").click()

                temp=dict()
                time.sleep(10)
                res1 = Selector(text=driver.page_source)
                address = res1.css('div.Io6YTe.fontBodyMedium::text').get()
                address1 = address.split(',')
#                 print('Address1: ', address1)
                if len(address1) > 3:
                    address1 = address1[1:]

                zipcode = address1[1].strip().split(" ")[0]
            #     zipcode = address1[1].split(" ")[0]

                town = address1[1].strip().split(" ")
#                 if len(town) == 2:
#                     town = town[0]
#                 else:
                town = " ".join(town[1:])
                street = address1[0]
                rating = res1.css('div.F7nice > span:nth-child(1) > span:nth-child(1) > span:nth-child(1)::text').get()
                total_reviews = res1.css('.F7nice button.DkEaL::text').get()


                print('Name: ', name)
                print('Street: ', street)
                print('Zip Code: ', zipcode)
                print('Town: ', town)
                print('Rating: ', rating)
                print('Total Reviews: ', total_reviews)

                temp = dict()
                temp['name'] = name
                temp['street'] = street
                temp['zipcode'] = zipcode
                temp['town'] = town
                temp['rating'] = rating
                temp['total_reviews'] = total_reviews
                list_of_dict.append(temp)
            except:
                try:
                    print('Name: ',name)
                    print('address: ',address)
                    driver.get('https://www.google.com/maps/')
                    time.sleep(3)
                    WebDriverWait(driver,10).until(EC.visibility_of_element_located((By.XPATH, "(//input[@class='tactile-searchbox-input'])[1]"))).send_keys("{} {}".format(name,address))
                    time.sleep(5)
                    driver.find_element_by_css_selector("div.DgCNMb").click()
#                     driver.find_element_by_css_selector("#searchbox-searchbutton").click()
                    temp=dict()
                    time.sleep(10)
                    restaurant_url = res1.css('a.hfpxzc::attr(href)').get()
                    print(restaurant_url)
                    driver.get(restaurant_url)
                    time.sleep(2)
                    res1 = Selector(text=driver.page_source)
                    address = res1.css('div.Io6YTe.fontBodyMedium::text').get()
                    address1 = address.split(',')
                    print('Address1: ', address1)
                    if len(address1) > 3:
                        address1 = address1[1:]

                    zipcode = address1[1].strip().split(" ")[0]
                #     zipcode = address1[1].split(" ")[0]

                    town = address1[1].strip().split(" ")
    #                 if len(town) == 2:
    #                     town = town[0]
    #                 else:
                    town = " ".join(town[1:])
                    street = address1[0]
                    rating = res1.css('div.F7nice > span:nth-child(1) > span:nth-child(1) > span:nth-child(1)::text').get()
                    total_reviews = res1.css('.F7nice button.DkEaL::text').get()


                    print('Name: ', name)
                    print('Street: ', street)
                    print('Zip Code: ', zipcode)
                    print('Town: ', town)
                    print('Rating: ', rating)
                    print('Total Reviews: ', total_reviews)

                    temp = dict()
                    temp['name'] = name
                    temp['street'] = street
                    temp['zipcode'] = zipcode
                    temp['town'] = town
                    temp['rating'] = rating
                    temp['total_reviews'] = total_reviews
                    list_of_dict.append(temp)
                
                except:
                    try:
                        print('Name: ',name)
                        print('address: ',address)

    #                     address = address.strip().split(" ")[0]
    #                     print('New address: ', address)
                        driver.get('https://www.google.com/maps/')
                        time.sleep(3)
                        WebDriverWait(driver,10).until(EC.visibility_of_element_located((By.XPATH, "(//input[@class='tactile-searchbox-input'])[1]"))).send_keys("{}".format(name))
                        time.sleep(5)
                        driver.find_element_by_css_selector("div.DgCNMb").click()
                        
                        temp=dict()
                        time.sleep(10)
                        res1 = Selector(text=driver.page_source)
                        address = res1.css('div.Io6YTe.fontBodyMedium::text').get()
                        address1 = address.split(',')
                        print('Address1: ', address1)
                        if len(address1) > 3:
                            address1 = address1[1:]

                        zipcode = address1[1].strip().split(" ")[0]
                #     zipcode = address1[1].split(" ")[0]

                        town = address1[1].strip().split(" ")
        #                 if len(town) == 2:
        #                     town = town[0]
        #                 else:
                        town = " ".join(town[1:])
                        street = address1[0]
                        rating = res1.css('div.F7nice > span:nth-child(1) > span:nth-child(1) > span:nth-child(1)::text').get()
                        total_reviews = res1.css('.F7nice button.DkEaL::text').get()


                        print('Name: ', name)
                        print('Street: ', street)
                        print('Zip Code: ', zipcode)
                        print('Town: ', town)
                        print('Rating: ', rating)
                        print('Total Reviews: ', total_reviews)

                        temp = dict()
                        temp['name'] = name
                        temp['street'] = street
                        temp['zipcode'] = zipcode
                        temp['town'] = town
                        temp['rating'] = rating
                        temp['total_reviews'] = total_reviews
                        list_of_dict.append(temp)
                    except:
                        try:
                            time.sleep(3)
                            restaurant_url = res1.css('a.hfpxzc::attr(href)').get()
                            print(restaurant_url)
                            driver.get(restaurant_url)
                            time.sleep(5)
                            temp=dict()

                            res1 = Selector(text=driver.page_source)
                            address = res1.css('div.Io6YTe.fontBodyMedium::text').get()
                            try:
                                address1 = address.split(',')
                            except:
                                address = res1.css('span.DkEaL::text').get()
                                address1 = address.split(',')
                            print('Address1: ', address1)
                            if len(address1) > 3:
                                address1 = address1[1:]

                            zipcode = address1[1].strip().split(" ")[0]
                        #     zipcode = address1[1].split(" ")[0]

                            town = address1[1].strip().split(" ")
            #                 if len(town) == 2:
            #                     town = town[0]
            #                 else:
                            town = " ".join(town[1:])
                            street = address1[0]
                            rating = res1.css('div.F7nice > span:nth-child(1) > span:nth-child(1) > span:nth-child(1)::text').get()
                            total_reviews = res1.css('.F7nice button.DkEaL::text').get()


                            print('Name: ', name)
                            print('Street: ', street)
                            print('Zip Code: ', zipcode)
                            print('Town: ', town)
                            print('Rating: ', rating)
                            print('Total Reviews: ', total_reviews)

                            temp = dict()
                            temp['name'] = name
                            temp['street'] = street
                            temp['zipcode'] = zipcode
                            temp['town'] = town
                            temp['rating'] = rating
                            temp['total_reviews'] = total_reviews
                            list_of_dict.append(temp)
                        
                        except:
                            print('Name: ',name)
                            print('address: ',address)
                            driver.get('https://www.google.com/maps/')
                            time.sleep(3)
                            WebDriverWait(driver,10).until(EC.visibility_of_element_located((By.XPATH, "(//input[@class='tactile-searchbox-input'])[1]"))).send_keys("{} {}".format(name,city))
                            time.sleep(5)
                            driver.find_element_by_css_selector("div.DgCNMb").click()
        #                     driver.find_element_by_css_selector("#searchbox-searchbutton").click()
                            temp=dict()
                            time.sleep(10)
#                             restaurant_url = res1.css('a.hfpxzc::attr(href)').get()
#                             print(restaurant_url)
#                             driver.get(restaurant_url)
#                             time.sleep(2)
                            res1 = Selector(text=driver.page_source)
                            address = res1.css('div.Io6YTe.fontBodyMedium::text').get()
                            address1 = address.split(',')
                            print('Address1: ', address1)
                            if len(address1) > 3:
                                address1 = address1[1:]

                            zipcode = address1[1].strip().split(" ")[0]
                        #     zipcode = address1[1].split(" ")[0]

                            town = address1[1].strip().split(" ")
            #                 if len(town) == 2:
            #                     town = town[0]
            #                 else:
                            town = " ".join(town[1:])
                            street = address1[0]
                            rating = res1.css('div.F7nice > span:nth-child(1) > span:nth-child(1) > span:nth-child(1)::text').get()
                            total_reviews = res1.css('.F7nice button.DkEaL::text').get()


                            print('Name: ', name)
                            print('Street: ', street)
                            print('Zip Code: ', zipcode)
                            print('Town: ', town)
                            print('Rating: ', rating)
                            print('Total Reviews: ', total_reviews)

                            temp = dict()
                            temp['name'] = name
                            temp['street'] = street
                            temp['zipcode'] = zipcode
                            temp['town'] = town
                            temp['rating'] = rating
                            temp['total_reviews'] = total_reviews
                            list_of_dict.append(temp)
                            
        print('Temp: ',temp)
    except:
        driver.find_element_by_css_selector('button.B7V4Ld').click()
        

In [ ]:
len(list_of_dict)
# list_of_dict = list_of_dict[:5]
# list_of_dict

In [ ]:
df = pd.DataFrame(list_of_dict)
df

In [ ]:
df.to_csv('data29.csv',index=False)

In [ ]:
df = pd.read_csv('berlin.csv')
df1 = pd.read_csv('data1.csv')
df2 = pd.read_csv('data2.csv')
df3 = pd.read_csv('data3.csv')
df4 = pd.read_csv('data4.csv')
df5 = pd.read_csv('data5.csv')
df6 = pd.read_csv('data6.csv')
df7 = pd.read_csv('data7.csv')
df8 = pd.read_csv('data8.csv')
df9 = pd.read_csv('data9.csv')
df10 = pd.read_csv('data10.csv')
df11 = pd.read_csv('data11.csv')
df12 = pd.read_csv('data12.csv')
df13 = pd.read_csv('data13.csv')
df14 = pd.read_csv('data14.csv')
df15 = pd.read_csv('data15.csv')
df16 = pd.read_csv('data16.csv')
df17 = pd.read_csv('data17.csv')
df18 = pd.read_csv('data18.csv')

In [ ]:
df

In [ ]:
new_df = pd.concat([df,df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11,df12,df13,df14,df15,df16,df17,df18])
new_df

In [ ]:
city = cities['cities'].tolist()[:20]
remove_rows = new_df[new_df['town'].isin(city)]
remove_rows = remove_rows.drop_duplicates(subset=['name','street'],keep='first')
remove_rows.groupby(['town'])['town'].count()


In [ ]:
new_df.drop_duplicates(subset=['name','street'],keep='first')

In [ ]:
new_df.to_csv('final_data.csv',index=False)

In [ ]:
final = pd.read_csv('new_data.csv')
# final['street'] = final['street'].str.replace('Mar','03').str.replace('Apr','04').str.replace('May','05').str.replace('Jun','06').str.replace('Jul','07').str.replace('Aug','08').str.replace('Sep','09').str.replace('Oct','10').str.replace('Nov','11').str.replace('Dec','12')
final = final.sort_values("name")
print(final)
final.to_excel('final_data.xlsx',index=False)

In [ ]:
final.head(30)

In [ ]:
df = pd.read_csv('data.csv')
df1 = pd.read_csv('data1.csv')
df2 = pd.read_csv('data2.csv')
df3 = pd.read_csv('data3.csv')
df4 = pd.read_csv('data4.csv')
df5 = pd.read_csv('data5.csv')
df6 = pd.read_csv('data6.csv')
df7 = pd.read_csv('data7.csv')



new_df1 = pd.concat([df,df1,df2,df3,df4,df5,df6,df7])
new_df1

In [ ]:
final_data = pd.read_csv('final_data.csv')
final_data

In [ ]:
final_data1 = pd.concat([final_data,new_df1])
final_data1

In [ ]:
final_data1 = final_data1[~final_data1.name.str.contains("Thalia")]
final_data1

In [ ]:
final_data1.to_csv('final_data1.csv',index=False)